In [2]:
# import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import csv
from datetime import datetime

#configuration
hostip = "192.168.100.4" #change me


def readCSVFile(fileName):
    list=[]
    with open('clickstream.csv', 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            list.append({'Clicks':int(row['Clicks']),'Impressions':int(row['Impressions'])})
    return list

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        #print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'clickstream'
    cRows = readCSVFile('clickstream.csv')
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    start_index=0
    
    rows_to_send=random.randint(5,10)
    late_rows_to_send=random.randint(1,3)
    late_data_to_send = []
    
    while True:        
        
        ts = {'ts': int(dt.datetime.now().timestamp())}
        to_send = cRows[start_index:start_index+rows_to_send]
        #increase the start index by the number of rows taken
        start_index = start_index+rows_to_send
        #append the timestamp into the object to be sent
        data=[dict(item,**ts) for item in to_send]        
        
        #late data
        to_send = cRows[start_index:start_index+late_rows_to_send]
        #increase the start index by the number of rows taken
        start_index = start_index+late_rows_to_send
        #append the timestamp into the object to be sent        
        late_data=[dict(item,**ts) for item in to_send]
            
        #publish both current and late data
        #the late data (from previous cycle is appended to the current data to be sent)
        data.extend(late_data_to_send)
        publish_message(producer, topic, data)
        
        #display the timestamps being sent
        for d in data:
            print(datetime.fromtimestamp(d['ts']),'impressions:',d['Impressions'])
        print('-------------------------')
        
        #after the current batch is sent, set the late data 
        late_data_to_send=late_data
        
        #reset to start from begining
        if(start_index>=len(cRows)):
            start_index=0
        sleep(5)
        

Publishing records..
2023-04-06 10:27:19 impressions: 3
2023-04-06 10:27:19 impressions: 3
2023-04-06 10:27:19 impressions: 3
2023-04-06 10:27:19 impressions: 3
2023-04-06 10:27:19 impressions: 11
2023-04-06 10:27:19 impressions: 11
2023-04-06 10:27:19 impressions: 7
2023-04-06 10:27:19 impressions: 5
-------------------------
2023-04-06 10:27:24 impressions: 4
2023-04-06 10:27:24 impressions: 8
2023-04-06 10:27:24 impressions: 3
2023-04-06 10:27:24 impressions: 4
2023-04-06 10:27:24 impressions: 6
2023-04-06 10:27:24 impressions: 5
2023-04-06 10:27:24 impressions: 6
2023-04-06 10:27:24 impressions: 3
2023-04-06 10:27:19 impressions: 3
-------------------------
2023-04-06 10:27:29 impressions: 4
2023-04-06 10:27:29 impressions: 5
2023-04-06 10:27:29 impressions: 4
2023-04-06 10:27:29 impressions: 6
2023-04-06 10:27:29 impressions: 7
2023-04-06 10:27:29 impressions: 2
2023-04-06 10:27:29 impressions: 4
2023-04-06 10:27:29 impressions: 4
2023-04-06 10:27:24 impressions: 5
---------------

2023-04-06 10:29:24 impressions: 7
2023-04-06 10:29:24 impressions: 3
2023-04-06 10:29:24 impressions: 11
2023-04-06 10:29:24 impressions: 4
2023-04-06 10:29:24 impressions: 6
2023-04-06 10:29:24 impressions: 4
2023-04-06 10:29:24 impressions: 2
2023-04-06 10:29:24 impressions: 10
2023-04-06 10:29:19 impressions: 4
-------------------------
2023-04-06 10:29:29 impressions: 1
2023-04-06 10:29:29 impressions: 2
2023-04-06 10:29:29 impressions: 3
2023-04-06 10:29:29 impressions: 5
2023-04-06 10:29:29 impressions: 5
2023-04-06 10:29:29 impressions: 9
2023-04-06 10:29:29 impressions: 3
2023-04-06 10:29:29 impressions: 6
2023-04-06 10:29:24 impressions: 5
-------------------------
2023-04-06 10:29:34 impressions: 5
2023-04-06 10:29:34 impressions: 3
2023-04-06 10:29:34 impressions: 7
2023-04-06 10:29:34 impressions: 3
2023-04-06 10:29:34 impressions: 6
2023-04-06 10:29:34 impressions: 5
2023-04-06 10:29:34 impressions: 7
2023-04-06 10:29:34 impressions: 7
2023-04-06 10:29:29 impressions: 5
-

KeyboardInterrupt: 